In [ ]:
from bs4 import BeautifulSoup as BS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/mdpi-20220819T182723Z-001.zip

mdpi_dir = '/content/mdpi'

# Get a list of all files in given directory
from os import walk
filenames = next(walk(mdpi_dir), (None, None, []))[2]  # [] if no file

Mounted at /content/drive
Archive:  /content/drive/MyDrive/mdpi-20220819T182723Z-001.zip
  inflating: mdpi/10.3390_rs90201023.html  
  inflating: mdpi/10.3390_su132111674.html  
  inflating: mdpi/10.3390_su12030932.html  
  inflating: mdpi/10.3390_rs9010047.html  
  inflating: mdpi/10.3390_HYDROLOGY7030048.html  
  inflating: mdpi/10.3390_rs13122352.html  
  inflating: mdpi/10.3390_f12091217.html  
  inflating: mdpi/10.3390_su10020472.html  
  inflating: mdpi/10.3390_su11010003.html  
  inflating: mdpi/10.3390_LAND7010029.html  
  inflating: mdpi/10.3390_fire4030059.html  
  inflating: mdpi/10.3390_rs11060709.html  
  inflating: mdpi/10.3390_rs13193910.html  
  inflating: mdpi/10.3390_su10093353.html  
  inflating: mdpi/10.3390_f12081063.html  
  inflating: mdpi/10.3390_su11164347.html  
  inflating: mdpi/10.3390_rs13183736.html  
  inflating: mdpi/10.3390_horticulturae4040044.html  
  inflating: mdpi/10.3390_rs9010048.html  
  inflating: mdpi/10.3390_su10114224.html  
  inflating: mdp

In [ ]:
import sklearn

In [ ]:
'''

## extract full text without references
text_div=soup.find_all("div", {"class": "html-body"})
for text in text_div:
  print(text.text)
'''


 1. IntroductionSilage is the main feed in ruminant meat and milk production in Northern Europe. In Finland, 23% of the cultivated area is used for silage production, which is approximately 513,500 ha. In the silage production, grass swards are harvested three times each season, and fertilizer is applied for each harvest. Timely information of the yield quality and quantity would be highly valuable for decision-making on harvesting time and rate of fertilizer application for the next harvest. In grass yield, the quantity increases rapidly in the spring growth. Simultaneously the quality decreases, especially in grass digestibility. Harvesting time optimization entails a balance between the highest possible yield quantity and an adequately high digestibility for feeding.The grass biomass has a strong correlation with canopy heights [1,2,3]. Therefore, the grass height is a fundamental parameter of interest when concerning precision management of grazing and silage harvesting. Accurate 

In [ ]:
'''


#separate text by sections, this will separate sub sections like section 2.1.
sections=text_div[0].find_all("section")
for sec in sections:
  print(sec.find_all('h2'))
  print(sec.text)
'''

In [ ]:
'''
#use find siblings to separate data only at first section level part1
first_sec=text_div[0].find("section")
print(first_sec)
print(first_sec.h2.text)
cleaned = first_sec.text.replace(first_sec.h2.text, "")
print(cleaned)
#print(first_sec.text) # note the h2 text is connected the first word of the paragraph
#print(len(first_sec.text))
'''

In [ ]:
'''
#use find siblings to separate data only at first section level part2
nextSiblings = first_sec.find_next_siblings("section")
print(type(nextSiblings))
# Printing all the next siblings
for nextSibling in nextSiblings:
  print(nextSibling)
  cleaned = nextSibling.text.replace(nextSibling.h2.text, "")
  print(cleaned)
  #print(nextSibling.text)
  print(nextSibling.h2.text)
'''

In [ ]:
#extract text data without h2 section title heads
import pandas as pd

#create text corpus
corpus = ""
text_rows = []
sect_titles = []
sub_sect_titles = []
class_labels = []
set_h2_names = set()


#Split into testing (10%), training (70%), and validation data (20%)

#start at file #0
file_num = 0

#set cutoffs for testing, training data
test_cutoff = int(0.1 * len(filenames))
train_cutoff = int(test_cutoff + (0.7 * len(filenames)))




#for each file, read it, parse, identify first section
for file_name in filenames:
    # There might be possible exceptions from extracting text. 
    # This will catch the exceptions and we can analyze why it failed for some files

  #read html file
  with open(mdpi_dir + '/' + file_name, "r", encoding='utf-8') as f:
          html_file = f.read()

  #soup parser
  soup=BS(html_file,'html.parser')

  #try-except block to catch errors
  try:
    text_div=soup.find_all("div", {"class": "html-body"})

    first_sec = text_div[0].find("section")

  except Exception as e:
    print("Error while extracting text for {}".format(file_name), e)

  #add cleaned text data, introduction section to list

  cleaned = first_sec.text.replace(first_sec.h2.text, "")
  corpus += cleaned
  text_rows.append(cleaned)
  sect_titles.append(first_sec.h2.text)
  set_h2_names.add(first_sec.h2.text)
  sub_sect_titles.append("None")
  class_labels.append(first_sec.h2.text.strip()[0:2])

  #Identify siblings for each file, save to CSV
  nextSiblings = first_sec.find_next_siblings("section")

  for nextSibling in nextSiblings:
    paras=nextSibling.find_all('section')
    
    if len(paras)==0:
      
      #get clean text, add section and subsection titles to list
      cleaned = nextSibling.text.replace(nextSibling.find(['h2', 'h3', 'h4']).text, "")
      #cleaned = cleaned.replace(para.find(['h2', 'h3', 'h4']).text, "")
      corpus += cleaned
      text_rows.append(cleaned)
      sect_titles.append(nextSibling.find(['h2', 'h3', 'h4']).text)
      sub_sect_titles.append("None")
      if (nextSibling.find(['h2', 'h3', 'h4']).text not in set_h2_names):
        set_h2_names.add(nextSibling.find(['h2', 'h3', 'h4']).text.strip()[3:].strip())
      class_labels.append(nextSibling.find(['h2', 'h3', 'h4']).text.strip()[:1].replace(".", ""))

    else: 
      for para in paras:

        #get clean text, add section and subsection titles to list
        cleaned = para.text.replace(para.find(['h2', 'h3', 'h4']).text, "")
        cleaned = cleaned.replace(nextSibling.find(['h2', 'h3', 'h4']).text, "")
        corpus += cleaned
        text_rows.append(cleaned)
        sub_sect_titles.append(para.find(['h2', 'h3', 'h4']).text)
        sect_titles.append(nextSibling.find(['h2', 'h3', 'h4']).text)
        if (nextSibling.find(['h2', 'h3', 'h4']).text not in set_h2_names):
          set_h2_names.add(nextSibling.find(['h2', 'h3', 'h4']).text[3:].strip())
        class_labels.append(nextSibling.find(['h2', 'h3', 'h4']).text.strip()[:1].replace(".", ""))

  #if at test cutoff, save to CSV, reset data values
  if (file_num == test_cutoff):
    d = {'text' : text_rows, 'Section Title' : sect_titles, "Subsection Titles" : sub_sect_titles, "Class Labels" : class_labels}
    df = pd.DataFrame(d)
    df.to_csv("Test_Data.csv")
    text_rows = []
    sect_titles = []
    sub_sect_titles = []
    class_labels = []

  #if at train cutoff, save to CSV, reset data values
  if (file_num == train_cutoff):
    d = {'text' : text_rows, 'Section Title' : sect_titles, "Subsection Titles" : sub_sect_titles, "Class Labels" : class_labels}
    df = pd.DataFrame(d)
    df.to_csv("Train_Data.csv")
    text_rows = []
    sect_titles = []
    sub_sect_titles = []
    class_labels = []

  #increment the file number
  file_num += 1

#Done with for loop, save data to validation csv

d = {'text' : text_rows, 'Section Title' : sect_titles, "Subsection Titles" : sub_sect_titles, "Class Labels" : class_labels}

df = pd.DataFrame(d)
df.to_csv("Validation_Data.csv")


Error while extracting text for 10.3390_rs90201024.html list index out of range
Error while extracting text for 10.3390_rs90201023.html list index out of range


In [ ]:
'''
print(len(set_h2_names))
list_h2_names = list(set_h2_names)
for title in list_h2_names:
  print(title)
'''
labels_set = set(class_labels)
print(len(labels_set))
print(labels_set)

7
{'2', '6', '7', '5', '4', '1.', '3'}


In [ ]:
#to do, create a dataframe to save the text data into csv as the following format

#column names:   Text,            |      Section_Title(h2),                       |        Subection_Title(h4) if any
#==============================================================================================================================
#             first_sec.text      |               first                           |        para.find(['h2','h3','h4']).text
#             nextSibling.text    |   nextSibling.find(['h2','h3','h4']).text     |        para.find(['h2','h3','h4']).text
#             para.text           |   nextSibling.find(['h2','h3','h4']).text     |        para.find(['h2','h3','h4']).text

In [ ]:
#import data back from CSV to DataFrame
import numpy as np

NB_train = pd.read_csv('Train_Data.csv')
SVM_train = pd.read_csv('Train_Data.csv')

NB_test = pd.read_csv('Test_Data.csv')
SVM_test = pd.read_csv('Test_Data.csv')

NB_train_text = NB_train["text"]
NB_train_labels = NB_train["Class Labels"]
NB_train_labels = NB_train_labels.astype(int)

SVM_train_text = SVM_train["text"]
SVM_train_labels = SVM_train["Class Labels"]
SVM_train_labels = SVM_train_labels.astype(int)
                       
NB_test_text = NB_test["text"]
NB_test_labels = NB_test["Class Labels"]
NB_test_labels = NB_test_labels.astype(int)

SVM_test_text = SVM_test["text"]
SVM_test_labels = SVM_test["Class Labels"]
SVM_test_labels = SVM_test_labels.astype(int)


#develop pipeline for Naive-Bayes

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
count_vect = CountVectorizer()

NB_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

NB_clf.fit(NB_train_text, NB_train_labels)

NB_predicted = NB_clf.predict(NB_test_text)

#develop pipeline for SVM model

from sklearn.linear_model import SGDClassifier
SVM_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None))])

SVM_clf.fit(SVM_train_text, SVM_train_labels)

SVM_predicted = SVM_clf.predict(SVM_test_text)

NB_accuracy = np.mean(NB_predicted == NB_test_labels)
SVM_accuracy = np.mean(SVM_predicted == SVM_test_labels)

print(NB_train_text.iloc[0])
print(SVM_train_text.iloc[0])

print("")
print(NB_test_labels.iloc[0:2])
print(SVM_test_labels.iloc[0:2])

print(SVM_clf.predict(SVM_test_text.iloc[0:2]))

print(type(NB_train_labels.iloc[0]))

print("Accuracy of NB model: ", NB_accuracy)
print("Accuracy of SVM model: ", SVM_accuracy)

ValueError: ignored

In [ ]:
df.iloc[0,
        :]

text                 The experiment was conducted at the LUKE resea...
Section Title                                                     None
Subsection Titles           2.1. Study Area and Reference Measurements
Name: 0, dtype: object

In [ ]:
print(df.iloc[0,0])

The experiment was conducted at the LUKE research farm, which is located in the municipality of Jokioinen in southwest Finland (approximately 60°48′ N, 23°30′ E) (Figure 1). The experiment was established on a second-year silage production field. The grass sward was established in 2015 in spring barley as a companion crop with a timothy/meadow fescue (Phleum pratense and Festuca pratensis) seed mixture at a 25 kg ha−1 sowing rate (65% timothy and 35% meadow fescue on a weight basis). The row width was 12.5 cm. In 2016, the field was managed as a silage production sward. A uniform and even site of the field was selected for the experiment. The soil type at the test site was clay, and the soil fertility values were as follows: pH 6.2, 377 mg K L−1 soil, 6.6 mg P L−1 soil, 1101 mg Mg L−1 soil, and 2580 mg Ca L−1 soil. The size of the experimental area was approximately 50 m by 20 m. The experimental setup was a split plot design with four replicates. The fertilizer treatment was in the 24

In [ ]:
type(df.iloc[0,0])

str

In [ ]:
print(df.iloc[0,0].replace(df.iloc[0,2],""))

The experiment was conducted at the LUKE research farm, which is located in the municipality of Jokioinen in southwest Finland (approximately 60°48′ N, 23°30′ E) (Figure 1). The experiment was established on a second-year silage production field. The grass sward was established in 2015 in spring barley as a companion crop with a timothy/meadow fescue (Phleum pratense and Festuca pratensis) seed mixture at a 25 kg ha−1 sowing rate (65% timothy and 35% meadow fescue on a weight basis). The row width was 12.5 cm. In 2016, the field was managed as a silage production sward. A uniform and even site of the field was selected for the experiment. The soil type at the test site was clay, and the soil fertility values were as follows: pH 6.2, 377 mg K L−1 soil, 6.6 mg P L−1 soil, 1101 mg Mg L−1 soil, and 2580 mg Ca L−1 soil. The size of the experimental area was approximately 50 m by 20 m. The experimental setup was a split plot design with four replicates. The fertilizer treatment was in the 24